In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
from __future__ import  absolute_import, unicode_literals, print_function, division

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Ver onsicolor', 'Virginica']
# Lets define some constants to help us later

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")


In [5]:
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe
train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header=0)

In [6]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [7]:
train_y = train.pop('Species')
test_y = test.pop('Species')
test.head() # the species column is now gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5
3,6.0,3.4,4.5,1.6
4,5.5,2.5,4.0,1.3


In [8]:
train.shape

(120, 4)

In [9]:
test.shape

(30, 4)

In [16]:
def input_fn(features, labels, training=True, batch_size = 256):
    #convert input intod dataset
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    if training: 
        #shuffle and repeat training data
        ds = ds.shuffle(100).repeat()
    
    return ds.batch(batch_size)


In [17]:
# Feature columns describe how to use the input
my_feature_columns = []

for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [18]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
clf = tf.estimator.DNNClassifier(
        feature_columns = my_feature_columns, 
        hidden_units = [30, 10], # 2 hidden layers of 30 and 10 units repsectively 
        n_classes = 3) # 3 classes in model

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Sanjay\\AppData\\Local\\Temp\\tmphjw9c1gr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
clf.train(
    input_fn = lambda: input_fn(train, train_y, training = True),
    steps = 5000)
    # We include a lambda to avoid creating an inner function previously
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Sanjay\AppData\Local\Temp\tmphjw9c1gr\model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\Sanjay\AppData\Local\Temp\tmphjw9c1gr\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.7622375, step = 5000
INFO:tensorflow:global_step/sec: 97.5463
INFO:tensorflow:loss = 0.7424493, step = 5100 (1.025 sec)
INFO:tensorflow:global_step/sec: 138.07
INFO:tensorflow:loss = 0.7457032, step = 5200 (0.724 sec)
INFO:tensorflow:global_step/sec: 168.228
INFO:tensorflow:loss = 0.75744706, step =

In [21]:
#evaluate on test set
eval_result = clf.evaluate(
    input_fn = lambda: input_fn(test, test_y, training = False))
print('Testing accuracy = {accuracy:0.4f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-15T16:27:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Sanjay\AppData\Local\Temp\tmphjw9c1gr\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.63989s
INFO:tensorflow:Finished evaluation at 2021-07-15-16:27:25
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.53333336, average_loss = 0.74373, global_step = 10000, loss = 0.74373
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\Sanjay\AppData\Local\Temp\tmphjw9c1gr\model.ckpt-10000
Testing accuracy = 0.5333



In [22]:
def input_fn(features, batch_size = 256):
    # Convert the inputs to a Dataset without labels.
    pred_set = tf.data.Dataset.from_tensor_slices(dict(features))
    return pred_set.batch(batch_size)

In [24]:
features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print('Type numeric values for the features as prompted')
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False
    
    predict[feature] = [float(val)]

predictions = clf.predict(input_fn = lambda: input_fn(predict))

for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print('Prediction is "{}" ({:.2f}%)'.format(SPECIES[class_id], probability*100))

Type numeric values for the features as prompted
SepalLength: 2.0
SepalWidth: 1.3
PetalLength: 0.87
PetalWidth: 4.6
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Sanjay\AppData\Local\Temp\tmphjw9c1gr\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 0.13167764, -0.9471754 ,  0.8253335 ], dtype=float32), 'probabilities': array([0.29931134, 0.10176144, 0.5989272 ], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (59.89%)
